In [1]:
import pickle
import pandas as pd
import numpy as np
import warnings
from sklearn.decomposition import NMF #비음수 행렬 분해 훈련에 사용
from sklearn.utils.extmath import randomized_svd # 특이값 분해 훈련에 사용

warnings.filterwarnings('ignore') #경고 메시지를 무시하고 숨기기

In [2]:
#요기요 크롤링 데이터셋 로드
df = pickle.load(open('/Users/giyeonlee/Downloads/추천시스템/추천시스템_프로젝트/Project_Final/YogingaDataSet.pkl','rb'))

In [3]:
print(df.tail())

         Restaurant UserID                                               Menu  \
23204  원할머니보쌈-상암배달점  in**님    맛있게 매운 의성마늘떡맵쌈/1(보쌈 사이즈 선택(중（생육710g이상,수육500g）))   
23205       장미로타리크럽  jk**님                      김.두.삼（김치두부삼겹살）/1,문어 초무침과 소면/1   
23206       장미로타리크럽  ja**님                                 문어삼합/1(사이즈 선택(3人))   
23207       장미로타리크럽     손님  유자보쌈（보쌈＋보쌈김치＋부추무침）/1(사이즈 선택(반근（300g）)),문어 초무침과...   
23208       장미로타리크럽  wd**님                                   통다리 문어버터구이와 땅콩/1   

                                                  Review Total Taste Quantity  \
23204  맛과 양 모두 만족스럽습니다! 맛있게 잘 먹겠습니다~ 배달도 빠르고 포장도 너무 깔...     5     5        5   
23205  배달이 좀 걸린듯 하지만 통두부에 삼겹김치.문어소면 맛있게 잘먹어요.폭풍흡입으로 사...     4     5        5   
23206                    여러번 먹었는데 맛있어요 사진이 없네요ㅠㅠ 호다닥 먹어서     5     5        5   
23207  친구네 놀러가서 시켜봤는데 진짜 전메뉴가 다 맛있다고 하는 친구말이 맞는거 같네요!...     5     5        4   
23208  그거알아요? 여기 메뉴 다맛있다는거☆ 항상오는 서비스 홍합탕 돈주고 시켜야되는거 아...     5     5        5   

      Delivery           D

In [4]:
df.rename(columns={'Total':'Rating'},inplace = True) #'Total 컬럼명을 'Rating'으로 변경
df = df.filter(['Restaurant', 'UserID', 'Rating']) #컬럼 'Restaurant', 'UserID', 'Rating'만 남기고 나머지 컬럼 삭제

print(df.tail())

         Restaurant UserID Rating
23204  원할머니보쌈-상암배달점  in**님      5
23205       장미로타리크럽  jk**님      4
23206       장미로타리크럽  ja**님      5
23207       장미로타리크럽     손님      5
23208       장미로타리크럽  wd**님      5


In [5]:
n_users = len(df['UserID'].unique()) #고유한 사용자 수
n_restaurants = len(df['Restaurant'].unique()) #고유한 삭당 수

n_users, n_restaurants

(1309, 635)

In [6]:
#평점 데이터에 대한 요약정보
#총 358132개의 평점
#평점은 1~5, nan 값으로로 총 6가지

df['Rating'].describe() 

count     358132
unique         6
top            5
freq      302892
Name: Rating, dtype: object

In [7]:
##전처리##

#예전 리뷰의 경우 사용자의 닉네임의 '손님'으로 통일되어 나오므로 제거
#UserID = '손님' 제거
#중복된 리뷰인 경우 최댓값으로 대치


#user-restaurant utility matrix
user_resaurant = df.pivot_table('Rating',index='UserID',columns = 'Restaurant',aggfunc='max')
user_resaurant.drop(user_resaurant.loc[['손님'],:].index, inplace = True)
print(user_resaurant)


#retaurant_user utility matrix
retaurant_user = df.pivot_table('Rating',index='Restaurant',columns = 'UserID',aggfunc='max')
retaurant_user.drop(retaurant_user[['손님']], inplace = True, axis = 1)
print(retaurant_user)



Restaurant      10평파스타&화덕피자 1인기사식당 1인살로만강명수아구찜앤탕-마포본점 1인용마라탕-서울수색점  \
UserID                                                               
              5         NaN    NaN                NaN          NaN   
--**님       NaN         NaN    NaN                NaN          NaN   
-0**님       NaN         NaN    NaN                NaN          NaN   
-9**님       NaN         NaN    NaN                NaN          NaN   
-_**님       NaN         NaN    NaN                NaN          NaN   
...         ...         ...    ...                ...          ...   
zv**님       NaN         NaN    NaN                NaN          NaN   
zw**님       NaN         NaN      5                NaN          NaN   
zx**님       NaN         NaN      4                NaN          NaN   
zy**님       NaN         NaN    NaN                NaN          NaN   
zz**님       NaN         NaN    NaN                NaN          NaN   

Restaurant 1인용묵은지김치찜-서울수색점 1인찌개와탕&냉면 1인한솥명가갈비찜-월곡점 1인할매보쌈족발-월곡점 24시 진짜루  ...  \
UserID   

In [8]:
#결측치 0으로 대체
retaurant_user.fillna(0,inplace = True)
user_resaurant.fillna(0,inplace = True)

print(retaurant_user.head())
print(user_resaurant.head())

UserID                --**님 -0**님 -9**님 -_**님 -b**님 -c**님 -e**님 -f**님 -g**님  \
Restaurant                                                                    
                    5     0     0     0     0     0     0     0     0     0   
10평파스타&화덕피자         0     0     0     0     0     0     0     0     0     0   
1인기사식당              0     0     0     0     0     0     0     0     0     0   
1인살로만강명수아구찜앤탕-마포본점  0     0     0     0     0     0     0     0     0     0   
1인용마라탕-서울수색점        0     0     0     0     0     0     0     0     0     0   

UserID              ... zq**님 zr**님 zs**님 zt**님 zu**님 zv**님 zw**님 zx**님 zy**님  \
Restaurant          ...                                                         
                    ...     0     0     0     0     0     0     0     0     0   
10평파스타&화덕피자         ...     5     0     0     0     0     0     0     0     0   
1인기사식당              ...     0     0     5     0     0     0     5     4     0   
1인살로만강명수아구찜앤탕-마포본점  ...     0     0     0

In [9]:
#학습을 위해 numpy로 변경
#float타입으로 변경
retaurant_user = retaurant_user.to_numpy().astype(np.float64)
retaurant_user

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 4., 0., 0.],
       ...,
       [0., 0., 0., ..., 5., 0., 5.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
##SVD 모델 학습##
"""
# sklearn에서 제공하는 randomized_svd()를 통해 SVD 모델을 학습하여 U, Sigma, VT를 구함
# 분해된 행렬 U, Sigma, VT의 형태는 (n_restaurants, k), (k,) (k, n_users)
# 분해된 행렬들을 이용하여 dot-product 연산을 수행하면 원래의 utility matrix와 같은 (n_restaurants, n_users) 형태의 근사 행렬을 얻을 수 있음
"""
k = 150 #embedding matrix의 feature 개수
U, Sigma, VT =  randomized_svd(retaurant_user, n_components = k,n_oversamples = 100, n_iter = 'auto') 
#n_oversamples는 M 범위를 샘플링하기 위한 추가 랜덤 벡터 수.
#오차를 줄이는 방향으로 n_oversamples 100으로 설정



# 분해된 행렬이 올바른 형태로 생성되었는지 확인
print(U.shape, Sigma.shape, VT.shape)

retaurant_user_approx_svd = np.dot(np.dot(U,np.diag(Sigma)),VT)

# 근사 행렬이 올바른 형태로 생성되었는지 확인
print(retaurant_user_approx_svd.shape)

(635, 150) (150,) (150, 1308)
(635, 1308)


In [11]:
pd.DataFrame(retaurant_user_approx_svd)

,0,1,2,3,4,5,6,7,8,9,...,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307
0,2.117394e-17,6.206667e-11,2.360620e-11,-2.844064e-11,2.998233e-10,-1.834157e-10,5.899269e-12,3.127255e-10,-4.631676e-11,-4.356350e-12,...,2.468220e-10,-2.235180e-10,-4.651255e-10,1.267685e-10,-2.580139e-10,-1.329881e-10,-3.250543e-10,-1.452284e-10,3.976612e-11,-8.255575e-11
1,3.123307e-09,1.849908e-01,-2.800989e-03,-3.046588e-02,2.918548e-01,6.652717e-01,1.724393e-02,-8.291340e-01,2.607610e-01,1.384667e-02,...,2.359710e+00,-1.359368e-01,5.568068e-01,-2.094690e-01,-8.716990e-01,1.008373e-01,2.194083e-01,1.315331e-01,-1.328802e-01,-1.742649e-01
2,-1.028408e-08,-2.059983e-02,-7.887338e-02,1.750516e-01,1.430840e-01,6.079417e-01,-1.514462e-01,4.201870e-01,5.037332e-02,6.953342e-02,...,-9.480259e-02,8.869238e-01,3.283904e+00,2.767673e-01,-6.186221e-01,3.671037e-01,2.521731e+00,3.234608e+00,2.536614e-01,2.176248e+00
3,5.390450e-10,2.560484e-02,2.095980e-02,-1.681011e-02,-9.719911e-02,-3.947835e-02,-5.534302e-02,1.473034e-01,-3.526835e-02,-7.210401e-02,...,-2.071474e-02,-1.027247e-01,1.241431e-01,-4.610891e-02,-1.557180e-01,3.196876e-02,-8.643306e-02,8.879745e-02,2.268473e-01,5.165815e-01
4,1.019247e-10,-3.846310e-02,-8.253289e-02,-1.454146e-03,-4.975652e-01,-8.912577e-02,-1.683979e-05,-4.080183e-01,6.344333e-02,-1.715862e-02,...,-3.467214e-01,3.049018e-01,1.408939e-01,9.226566e-03,-8.465988e-02,3.386311e-05,-5.055306e-02,7.731643e-02,-9.859847e-02,2.603816e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,-4.538308e-09,-2.335469e-02,-9.250839e-02,9.901391e-03,5.604604e-01,1.359654e+00,-1.435438e-02,-2.525045e-01,9.219921e-02,2.874930e-04,...,1.885242e-01,4.120534e-01,-4.329892e-01,6.116850e-02,-3.210702e-01,-9.757541e-02,1.192910e-01,-9.774386e-02,-2.403402e-01,-3.330581e-01
631,-8.004190e-10,9.471837e-02,1.052268e-01,7.490758e-02,-2.369196e-01,1.829406e-02,-5.762071e-01,-1.180631e-01,-1.528305e-01,1.288849e-01,...,6.314387e-01,5.238629e-01,1.703022e-01,-5.806292e-02,5.906023e-01,3.512454e-01,1.473076e-01,6.539745e-01,6.730030e-01,-2.314965e-01
632,2.297963e-09,-6.609555e-02,1.031694e-01,4.631962e-02,5.247055e+00,-3.702942e-01,-9.162096e-02,5.093333e+00,-3.120737e-02,2.275583e-01,...,2.450397e-01,-2.372777e-01,4.976254e+00,2.604363e-01,4.371019e+00,1.262329e-01,2.959042e-01,4.544475e+00,7.325325e-01,5.755202e+00
633,3.232164e-10,4.885474e-02,1.033200e-01,-2.540526e-03,-3.297224e-01,3.920555e-01,2.162325e-02,2.368502e-01,7.953294e-02,5.688426e-02,...,7.747565e-02,-3.108587e-01,-8.982744e-01,1.312250e-01,8.634021e-01,-4.716070e-02,-3.459732e-01,1.689296e-01,3.067320e-01,2.919684e-01


In [12]:
##NMF 모델 학습##
"""
# sklearn에서 제공하는 NMF()를 통해 NMF 모델을 학습하여 W, H를 구함
# W와 H는 분해된 행렬,  W와 H의 형태는 (n_restaurants, k), (k, n_users)
# 분해된 행렬들을 dot-product 연산을 수행하면 원래의 utility matrix와 같은 (n_restaurants, n_users) 형태의 근사 행렬을 구할 수 있음
"""

k = 150 #embedding matrix의 feature 개수
model_nmf = NMF(n_components = k, init = 'random')
model_nmf.fit(retaurant_user)
W = model_nmf.transform(retaurant_user)
H = model_nmf.components_

# 분해된 행렬이 올바른 형태로 생성되었는지 확인
print(W.shape, H.shape)

retaurant_user_approx_nmf = np.dot(W,H)

# 근사 행렬이 올바른 형태로 생성되었는지 확인
print(retaurant_user_approx_nmf.shape)

(635, 150) (150, 1308)
(635, 1308)


In [13]:
##loss 계산 함수##
# 실제 utility matrix 와 SVD,NMF 를 통해 생성된 행렬 간의 오차를 계산

def compute_error(actual, prediction):
    # 매개변수로 입력받은 actual 행렬 안의 0값을 갖는 원소들은 오차 계산에서 제외
    prediction = prediction[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    sse = np.sum((actual - prediction) ** 2)
    rmse = np.sqrt((sse / len(actual)))
    
    return sse, rmse

def compute_error_all(actual, prediction):
    # actual 행렬 안의 0값을 갖는 원소들도 포함해서 오차를 계산
    sse = np.sum((actual - prediction) ** 2)
    rmse = np.sqrt((sse / len(actual)))
    
    return sse, rmse

In [14]:
#SVD모델이 NMF모델보다 오차률이 작음
print(f"SVD Error(ignoring zero values): SSE = {compute_error(retaurant_user, retaurant_user_approx_svd)[0]}, RMSE = {compute_error(retaurant_user, retaurant_user_approx_svd)[1]}")
print(f"NMF Error(ignoring zero values): SSE = {compute_error(retaurant_user, retaurant_user_approx_nmf)[0]}, RMSE = {compute_error(retaurant_user, retaurant_user_approx_nmf)[1]}")

print('\n')

print(f"SVD Error(including all zero values): SSE = {compute_error_all(retaurant_user, retaurant_user_approx_svd)[0]}, RMSE = {compute_error_all(retaurant_user, retaurant_user_approx_svd)[1]}")
print(f"NMF Error(including all zero values): SSE = {compute_error_all(retaurant_user, retaurant_user_approx_nmf)[0]}, RMSE = {compute_error_all(retaurant_user, retaurant_user_approx_nmf)[1]}")



SVD Error(ignoring zero values): SSE = 121049.20341906238, RMSE = 1.3421553726608435
NMF Error(ignoring zero values): SSE = 266719.5294517113, RMSE = 1.9922747668345202


SVD Error(including all zero values): SSE = 230917.23878981886, RMSE = 19.069588221684196
NMF Error(including all zero values): SSE = 372346.6444925455, RMSE = 24.215133047171097


In [15]:
##특정 사용자에 대한 유틸리티 매트릭스 retaurant_user에서 평점이 매겨지지 않은 값 예측##

def makePredictions(actual, pred, user):
    #rated_restaurant, unrated_restaurant 생성을 하기 위한 사전작업 
    user -= 1
    rated_items_df = pd.DataFrame(actual).iloc[:, user] #실제 df에서 user열 전체 선택
    user_pred_df = pd.DataFrame(pred).iloc[:,user] #예측 df에서 user열 전체 선택
    df_c = pd.DataFrame(df['Restaurant'].unique()) #returant의 이름 갖는 열이 1개인 dataframe
    total_df = pd.concat([rated_items_df, user_pred_df, df_c], axis = 1) #rated_items_df,user_pred_df,df_c를 왼쪽 + 오른쪽으로 합치기
    total_df.columns = ['Rating', 'prediction', 'Restaurant'] #total_df의 칼럼명을 Rating,prediction,Restaurant로 생성
    print(total_df)
    
    # 실제로 먹은 식당 목록 (평점이 부여된 식당 목록)
    rated_restaurant = total_df.where(total_df['Rating']>0)
    rated_restaurant= rated_restaurant.dropna()
    rated_restaurant= rated_restaurant.sort_values(by='Rating', ascending = False)
    
    # 예측할 식당 목록 (평점이 부여되지 않은 식당 목록)
    unrated_restaurant = total_df[total_df['Rating'] == 0]
    unrated_restaurant = unrated_restaurant.sort_values(by='prediction', ascending = False)
    
    return rated_restaurant, unrated_restaurant


In [26]:
# 특정 사용자에 대한 예측 결과
userNumber = 0

prediction_with_rated_svd, prediction_with_unrated_svd = makePredictions(retaurant_user, retaurant_user_approx_svd, userNumber) # 실제 Utility Matrix와 svd를 통해 근사한 행렬 간의 비교
prediction_with_rated_nmf, prediction_with_unrated_nmf = makePredictions(retaurant_user, retaurant_user_approx_nmf, userNumber) # 실제 Utility Matrix와 nmf를 통해 근사한 행렬 간의 비교

     Rating    prediction      Restaurant
0       0.0 -8.255575e-11       꽃게나라 간장게장
1       0.0 -1.742649e-01     키싸-디저트&눈꽃빙수
2       0.0  2.176248e+00       미스터피자-상암점
3       0.0  5.165815e-01          쿠파스테이크
4       0.0  2.603816e-01      프랭크버거-광흥창점
..      ...           ...             ...
630     0.0 -3.330581e-01  하늘호수보쌈족발-합정망원점
631     0.0 -2.314965e-01    호세야오리바베큐-합정점
632     5.0  5.755202e+00      원할머니보쌈-망원점
633     0.0  2.919684e-01      매일삶는족발-서교점
634     0.0 -1.356101e-01    원할머니보쌈-상암배달점

[635 rows x 3 columns]
     Rating  prediction      Restaurant
0       0.0    0.000000       꽃게나라 간장게장
1       0.0    0.432595     키싸-디저트&눈꽃빙수
2       0.0    2.997640       미스터피자-상암점
3       0.0    0.191341          쿠파스테이크
4       0.0    0.389722      프랭크버거-광흥창점
..      ...         ...             ...
630     0.0    0.312052  하늘호수보쌈족발-합정망원점
631     0.0    2.758999    호세야오리바베큐-합정점
632     5.0    4.991513      원할머니보쌈-망원점
633     0.0    0.685905      매일삶는족발-서교점
634     0.0    0.129684    원할머니보

In [27]:
prediction_with_rated_svd.head(10) #실제평점과 예측 평점 비교 - svd

,Rating,prediction,Restaurant
9,5.0,4.589382,NAPALPIZZA&WINE
473,5.0,3.996187,티바두마리치킨-망원점
435,5.0,3.687097,세꼬시곳간
449,5.0,4.604690,소문난간이횟집
454,5.0,5.688909,순살몬스터-신수마을점
455,5.0,5.283721,청년치킨-서교점
461,5.0,5.223943,지코바치킨-홍대점
465,5.0,5.612278,네네치킨-서교점
467,5.0,5.178581,동근이숯불두마리치킨-망원점
468,5.0,5.369337,페리카나-합정동점


In [28]:
prediction_with_rated_nmf.head(10) #실제평점과 예측 평점 비교 - nmf

,Rating,prediction,Restaurant
9,5.0,5.081879,NAPALPIZZA&WINE
473,5.0,2.027178,티바두마리치킨-망원점
435,5.0,1.659618,세꼬시곳간
449,5.0,5.162128,소문난간이횟집
454,5.0,3.434117,순살몬스터-신수마을점
455,5.0,4.383234,청년치킨-서교점
461,5.0,4.782951,지코바치킨-홍대점
465,5.0,5.163567,네네치킨-서교점
467,5.0,5.241005,동근이숯불두마리치킨-망원점
468,5.0,5.094670,페리카나-합정동점


In [29]:
prediction_with_unrated_svd.head(10) #예측 평점이 높은 순으로 새로운 식당 10개 식당 추천 - svd

,Rating,prediction,Restaurant
603,0.0,2.263789,꼬치꼬치양꼬치
2,0.0,2.176248,미스터피자-상암점
257,0.0,2.144737,명태요리전문점김명태-서교점
444,0.0,2.032146,신촌돈부리모노-본점
464,0.0,1.985538,푸라닭-망원점
590,0.0,1.786323,식공간-연남점
394,0.0,1.654453,이불덮밥-상암DMC점
470,0.0,1.593873,로타리치킨홍대상회
282,0.0,1.412486,"상수,곱도리탕"
81,0.0,1.407305,쿠벤일식도시락


In [30]:
prediction_with_unrated_nmf.head(10) #예측 평점이 높은 순으로 새로운 식당 10개 식당 추천 - nmf

,Rating,prediction,Restaurant
140,0.0,3.340058,연희옥-연희본점
257,0.0,3.283451,명태요리전문점김명태-서교점
2,0.0,2.997640,미스터피자-상암점
238,0.0,2.798416,신촌해물떡찜
504,0.0,2.759761,처갓집양념치킨-홍대점
631,0.0,2.758999,호세야오리바베큐-합정점
40,0.0,2.742423,식스티즈버거
237,0.0,2.725164,삼겹펀치-마포본점
83,0.0,2.645004,USA존슨부대찌개-수색점
329,0.0,2.345339,동대문엽기떡볶이-신촌점


In [22]:
#결과
# SVD 오차(RMSE) = 1.3421553726608433
# NMF 오차(RMSE) = 1.9955236769697513
# 으로 SVD모델의 성능이 더 좋음
# 특정 사용자를 선택하여 예측값이 높은순으로 새로운 식당을 추천해줄 수 있음